In [ ]:
!python --version


Python 3.10.12


In [ ]:
#install the dependencies
!pip install openai==0.28
import openai

In [ ]:
openai.api_key = ''


In [ ]:
loganFace = "https://dl.dropboxusercontent.com/scl/fi/krbbp4mw5jb9734qk0rk7/Logan-Face.png?rlkey=yils0zd54vnyi4rwepomgzthz&st=yc863roc&dl=0"
einsteinFace = "https://dl.dropboxusercontent.com/scl/fi/vgko9watd31oryaylzrn6/Screenshot-2024-10-08-142213.png?rlkey=evp8f8ixx5npfm1913fyc6jiq&st=hdm5wujp&dl=0"

In [ ]:

def generate_spirit_animal_description(urlInput):
  completion = openai.ChatCompletion.create(
      model="gpt-4o",
      messages=[
          {
              "role": "user",
              "content": [
                  {"type": "text", "text": "what would be a good spirit animal for him? Give me a one word response for the animal without a period"},
                  {
                      "type": "image_url",
                      "image_url": {
                          "url": urlInput,
                      }
                  },
              ],
          }
      ],
  )

  return completion.choices[0].message["content"]

In [ ]:
#this will be the main function

# Generate the spirit animal description
spirit_animal_logan = generate_spirit_animal_description(loganFace)
spirit_animal_einstein = generate_spirit_animal_description(einsteinFace)

# Print the animal from the output
print(f"Spirit Animal for Logan: {spirit_animal_logan}")
print(f"Spirit Animal for Einstein: {spirit_animal_einstein}")

Spirit Animal for Logan: Chameleon
Spirit Animal for Einstein: Owl


# Pose Detection

In [ ]:

!pip install controlnet-aux
!pip install diffusers transformers accelerate scipy safetensors
import cv2
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import numpy as np
from diffusers.utils import load_image
import matplotlib.pyplot as plt
from controlnet_aux import OpenposeDetector

def generate_animal_image(text):
  spirit_animal_einstein = text

  # Load and preprocess the image
  openpose = OpenposeDetector.from_pretrained('lllyasviel/ControlNet')
  image = load_image(loganFace)
  image = openpose(image)

  # Enable GPU and half-precision
  device = torch.device("cuda")
  controlnet = ControlNetModel.from_pretrained(
      "lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16
  ).to(device)

  pipe = StableDiffusionControlNetPipeline.from_pretrained(
      "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
  ).to(device)

  pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

  # Generate the image on GPU
  with torch.autocast("cuda"):
      image = pipe(
          f"a {spirit_animal_einstein} in a similar head-and-shoulder pose as the original image. Make sure the generated image includes the animal's neck and shoulders if it has one",
          image,
          num_inference_steps=50
      ).images[0]

  # Display the image
  plt.figure(figsize=(8, 8))
  plt.imshow(image)
  plt.axis('off')
  plt.show()

  image.save("animal_unedited.png", format="PNG")
  return image


/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [ ]:
# image.save("animal.png")

This will download the image

In [ ]:
!pip install requests
import requests

def download_dropbox_image(dropbox_url, output_filename="output.png"):
    # Ensure the link is set for direct download
    direct_url = dropbox_url.replace("?dl=0", "?dl=1")

    # Download the image
    response = requests.get(direct_url, stream=True)
    if response.status_code == 200:
        # Save the image as a .png file
        with open(output_filename, 'wb') as f:
            f.write(response.content)
        print(f"Image successfully downloaded and saved as {output_filename}")
    else:
        print("Failed to download the image. Check the URL and try again.")

download_dropbox_image(loganFace, "Logan Face.png")
download_dropbox_image(einsteinFace, "Einstein Face.png")

Image successfully downloaded and saved as Logan Face.png
Image successfully downloaded and saved as Einstein Face.png


In [ ]:
!pip install rembg
!pip install onnxruntime
from rembg import remove
def extract_background(image):
    # Load the original image
    original_image = image.convert("RGBA")
    # Image.open(image_path).convert("RGBA")  # Ensure it's RGBA format

    # Use rembg to remove the person (subject)
    removed_person = remove(original_image)

    # Convert the output to a NumPy array for processing
    removed_person_np = np.array(removed_person)

    # Create a mask from the alpha channel (transparent areas will be background)
    alpha_channel = removed_person_np[:, :, 3]  # Extract the alpha channel
    person_mask = (alpha_channel > 0).astype(np.uint8)  # Create a binary mask where person is 1, background is 0

    # Invert the mask to get the background-only mask
    background_mask = 1 - person_mask

    # Convert the original image to a NumPy array
    original_image_np = np.array(original_image)[:, :, :3]  # Discard the alpha channel

    # Apply the background mask to the original image
    background_only = original_image_np * background_mask[:, :, np.newaxis]

    # Convert back to an image and save the result
    result_image = Image.fromarray(background_only.astype(np.uint8))
    return result_image
    #result_image.save(output_path)

    # Display the result
    plt.imshow(result_image)
    plt.axis('off')
    plt.title('Background Only')
    plt.show()

# Usage

#extract_background('Logan Face.png', 'background_only.png')

In [ ]:
from rembg import remove
from PIL import Image
import numpy as np

def create_person_mask(input_image):
    # Load the image
    # input_image = Image.open(input_image_path)

    # Use rembg to remove the background
    no_bg_image = remove(input_image)

    # Convert the image to grayscale and threshold it to create a binary mask
    no_bg_image = no_bg_image.convert("L")
    mask = np.array(no_bg_image)

    # Set pixels where the person is present (non-transparent) to white (255)
    # and background to black (0)
    mask = np.where(mask > 0, 255, 0).astype(np.uint8)

    # Convert the mask back to an image
    mask_image = Image.fromarray(mask)
    return mask_image

    # # Save the mask image
    # mask_image.save(output_mask_path)
    # print(f"Mask saved to {output_mask_path}")

# Example usage
# create_person_mask("Logan Face.png", "output_mask.png")


we wil use deepfillv2 to fill out the background

### Fill out the background

In [ ]:
!pip install simple-lama-inpainting

In [ ]:
from simple_lama_inpainting import SimpleLama
from PIL import Image
def inpaint_background(image, mask):
  simple_lama = SimpleLama()

  # img_path = "background_only.png"
  # mask_path = "output_mask.png"

  # image = Image.open(img_path)
  # mask = Image.open(mask_path).convert('L')

  result = simple_lama(image, mask)
  return result
  # result.save("inpainted.png")

In [ ]:
from rembg import remove
from PIL import Image
import io

# Open the image file
input_path = 'animal_unedited.png'

def remove_background(input):
  # Use rembg to remove the background
  output_data = remove(input)

  # Convert the output data to an image and save
  # output_image = Image.open(io.BytesIO(output_data))
  return output_data
  # output_image.save('output_image.png')


In [ ]:
from PIL import Image

def overlay_foreground_on_background(foreground, background, output_path):
    foreground = foreground.convert("RGBA")  # Ensure it's in RGBA for transparency
    background = background.convert("RGBA")  # Ensure background is also RGBA

    # Resize background to match the size of the foreground if they are not the same size
    background = background.resize(foreground.size)

    # Composite the images: paste foreground on top of background
    combined_image = Image.alpha_composite(background, foreground)

    # Save the result
    combined_image.save(output_path, format="PNG")

    # Optionally display the result
    combined_image.show()
    return combined_image

    # print(f"Image saved as {output_path}")

# Example usage:
# overlay_foreground_on_background("output_image.png", "inpainted.png", "final_output.png")

In [ ]:
# install if needed
!pip install --upgrade gradio

  Using cached gradio-5.8.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.5.1-py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
 

In [ ]:
url = "https://drive.google.com/uc?id=1-zJgcQ9YMlsoWQG4X_UDdHKaV-ekbj9n"
print(generate_spirit_animal_description(url))

Owl


In [ ]:
!pip install torchsr

In [ ]:
#this will upscale the image
import torch
from torchvision.transforms.functional import to_pil_image, to_tensor
from torchsr.models import edsr_baseline
from PIL import Image

def upscale_image(image_path):
# Load your low-resolution image
  img = Image.open(image_path).convert("RGB")

  # Convert image to tensor and add batch dimension
  lr = to_tensor(img).unsqueeze(0)

  # Load the pretrained EDSR model for 2x upscaling
  model = edsr_baseline(scale=4, pretrained=True)

  # Ensure the model is in evaluation mode
  model.eval()

  # Perform super-resolution
  with torch.no_grad():
      sr = model(lr)

  # Remove batch dimension and convert back to PIL image
  sr_image = to_pil_image(sr.squeeze(0).clamp(0, 1))

  # Save or display the upscaled image
  return sr_image


In [ ]:
#this code will apply an art style to the image
from diffusers import StableDiffusionImg2ImgPipeline
#this code will apply an art style to the image
def apply_style_transfer(image, prompt, output_path="styled_image.png"):
    # Load the model (Ensure you have a pre-trained Stable Diffusion model)
    model_id = "runwayml/stable-diffusion-v1-5"  # Replace with a custom model for specific styles if needed
    # def disable_safety_checker(pipe):
    #   def dummy_checker(images, **kwargs):
    #       return images, [False] * len(images)
    #   pipe.safety_checker = dummy_checker

    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
    # disable_safety_checker(pipe)
    # Load input image
    if not isinstance(image, Image.Image):
      if image.startswith("http"):
        image = Image.open(requests.get(image, stream=True).raw).convert("RGB")

    # Style transfer settings
    guidance_scale = 7.5  # Controls the strength of the prompt
    num_inference_steps = 50  # Controls the quality

    # Apply style transfer
    styled_image = pipe(
        prompt=prompt,
        image=image,
        strength=0.7,  # Controls how much the output leans toward the prompt vs the original image
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps
    ).images[0]

    # Save the result
    styled_image.save(output_path)

    return styled_image


In [ ]:
auth.authenticate_user()
drive_service = build('drive', 'v3')

def upload_to_drive(file_path):
    file_name = os.path.basename(file_path)
    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype='image/png')

    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file['id']

def generate_shareable_link(file_id):
    drive_service.permissions().create(
        fileId=file_id,
        body={'role': 'reader', 'type': 'anyone'},
    ).execute()
    shareable_link = f"https://drive.google.com/uc?id={file_id}"
    # print(f"Generated URL: {shareable_link}")
    return shareable_link


def spirit_animal_main(image):
    image_path = "temp_image.png"
    image.save(image_path)
    file_id = upload_to_drive(image_path)
    image_url = generate_shareable_link(file_id)
    print(image_url)
    print(generate_spirit_animal_description(image_url))
    spirit_animal_text = generate_spirit_animal_description(image_url)
    generated_image = generate_animal_image(spirit_animal_text)

    # additional background removal
    mask = create_person_mask(image) # using original person image
    output_image = remove_background(generated_image) # no background spirit naimal
    inpainted_image = inpaint_background(image, mask) # using original person image
    final_image = overlay_foreground_on_background(output_image, inpainted_image, "outputImage")

    return spirit_animal_text, final_image

def apply_style_main(image):
    image_path = "temp_image1.png"
    image.save(image_path)
    file_id = upload_to_drive(image_path)
    image_url = generate_shareable_link(file_id)
    print(image_url)
    output = apply_style_transfer(image_url, "a surreal painting in the style of Van Gogh")
    return output


def main(image, action, prompt=None):
    _, sp_animal = spirit_animal_main(image)
    output = apply_style_main(sp_animal)
    return "Result Ready!", sp_animal, output

iface = gr.Interface(
    fn=main,
    inputs=[
        gr.Image(type="pil", label="Upload Your Image"),
    ],
    outputs=[
        gr.Textbox(label="Result Description"),
        gr.Image(label="Spirit Image"),
        gr.Image(label="Styled Image")
    ],
    title="Image Transformation Tool",
    description="Choose an action: generate a spirit animal or apply a style transfer to your image."
)

iface.launch(debug=True)

# iface = gr.Interface(
#     fn=main,
#     inputs=gr.Image(type="pil", label="Upload Your Image"),
#     outputs=[
#         gr.Textbox(label="Spirit Animal Description"),
#         gr.Image(label="Generated Animal Image")
#     ],
#     title="Spirit Animal Generator",
#     description="Upload an image or provide an image URL to discover your spirit animal and generate an image of it."
# )

# iface.launch(debug=True)

NameError: name 'auth' is not defined

In [ ]:
# version 2

import gradio as gr
import os
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
auth.authenticate_user()
drive_service = build('drive', 'v3')

def upload_to_drive(file_path):
    file_name = os.path.basename(file_path)
    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype='image/png')

    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file['id']

def generate_shareable_link(file_id):
    drive_service.permissions().create(
        fileId=file_id,
        body={'role': 'reader', 'type': 'anyone'},
    ).execute()
    shareable_link = f"https://drive.google.com/uc?id={file_id}"
    # print(f"Generated URL: {shareable_link}")
    return shareable_link

def new_style(image, prompt):
    model_id = "runwayml/stable-diffusion-v1-5"  # Pre-trained model
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
    styled_image = pipe(
        prompt=prompt,
        image=image,
        strength=0.7,
        guidance_scale=7.5,
        num_inference_steps=50
    ).images[0]

    # Return the styled image directly
    return styled_image


def process_image(image, mode, style_prompt):
    image_path = "temp_image.png"
    image.save(image_path)
    file_id = upload_to_drive(image_path)
    image_url = generate_shareable_link(file_id)
    spirit_animal_text = generate_spirit_animal_description(image_url)
    generated_image = generate_animal_image(spirit_animal_text)

    mask = create_person_mask(image) # using original person image
    output_image = remove_background(generated_image) # no background spirit naimal
    inpainted_image = inpaint_background(image, mask) # using original person image
    final_image = overlay_foreground_on_background(output_image, inpainted_image, "output_image")

    styled_image = new_style(final_image, style_prompt)
    return "Stylized Image", final_image, styled_image

iface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="pil", label="Upload Your Image"),
        gr.Radio(choices=["Spirit Animal", "Stylized Image"], label="Choose Image Processing Mode"),
        gr.Textbox(label="Style Prompt", value="a surreal painting in the style of Van Gogh")
    ],
    outputs=[
        gr.Textbox(label="Description/Result"),
        gr.Image(label="Spirit Animal"),
        gr.Image(label="Stylized Image")
    ],
    title="Image Processing Tool",
    description="Upload an image to either discover your spirit animal or apply a stylized art transformation."
)

iface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bd4bc5b9100ed83bd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


body_pose_model.pth:   0%|          | 0.00/209M [00:00<?, ?B/s]

hand_pose_model.pth:   0%|          | 0.00/147M [00:00<?, ?B/s]

facenet.pth:   0%|          | 0.00/154M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

safety_checker/config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

In [ ]:
!gradio deploy